In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

# Load data
df = pd.read_csv('Bruska_data.csv')
df.dropna(inplace=True)
df.drop(["time stamp", "Bruska_normally operating number"], axis=1, inplace=True)
print(df.shape)
# Data preprocessing
scaler_X = MinMaxScaler(feature_range=(0,1))
scaler_y = MinMaxScaler(feature_range=(0,1))
Xs = scaler_X.fit_transform(df.drop(['Bruska_active power output'], axis=1))
ys = scaler_y.fit_transform(df[['Bruska_active power output']])

(17189, 7)


In [2]:
window= 1
X=[]
Y=[]
for i in range(window,len(Xs)):
    X.append(Xs[i-window:i,:])
    Y.append(ys[i])
X,Y = np.array(X), np.array(Y)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [4]:
X_train.shape, X_test.shape

((13750, 1, 6), (3438, 1, 6))

In [5]:
# Define Informer model
import tensorflow as tf
from tensorflow.keras import layers, Model

# Define Encoder layer in the Informer model
class EncoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation='relu'),  # Changed activation to relu
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training=None, mask=None):
        attn_output = self.mha(x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        # Apply feedforward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        # Ensure ffn_output has the same shape as out1
        ffn_output = ffn_output[:, :, :x.shape[-1]]  # Trim ffn_output if necessary
        
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

# Define Encoder layer in the Informer model
class Encoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

    def call(self, x, training=None, mask=None):
        for enc_layer in self.enc_layers:
            x = enc_layer(x, training=training, mask=mask)
        return x

# Define Informer model
class Informer(Model):
    def __init__(self, input_dim, output_dim, num_encoder_layers, d_model, num_heads, dff, maximum_position_encoding, rate=0.1):
        super(Informer, self).__init__()
        self.encoder = Encoder(num_encoder_layers, d_model, num_heads, dff, maximum_position_encoding, rate)
        self.final_layer = layers.Dense(output_dim)

    def call(self, inputs, training=None):
        x = self.encoder(inputs, training=training)
        x = self.final_layer(x[:, -1, :])  # Extract the last timestep's prediction
        return x


input_dim = 6  # Dimension of input features
output_dim = 1  # Dimension of output predictions
num_encoder_layers = 4  # Number of encoder layers in the model
d_model = 32  # Dimension of the model
num_heads =8  # Number of attention heads
dff = 32  # Dimension of feedforward network
maximum_position_encoding = 10000  # Maximum position encoding
rate = 0.1 # Dropout rate
# Instantiate Informer model
informer_model = Informer(
    input_dim=input_dim,
    output_dim=output_dim,
    num_encoder_layers=num_encoder_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    maximum_position_encoding=maximum_position_encoding,
    rate=rate
)

# Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
informer_model.compile(optimizer=optimizer, loss='mean_absolute_error')

# Train the model
history = informer_model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss = informer_model.evaluate(X_test, y_test)

# Make predictions
predictions = informer_model.predict(X_test)


Epoch 1/100
430/430 [==============================] - 14s 10ms/step - loss: 0.1616
Epoch 2/100
430/430 [==============================] - 5s 11ms/step - loss: 0.0979
Epoch 3/100
430/430 [==============================] - 5s 13ms/step - loss: 0.0917
Epoch 4/100
430/430 [==============================] - 4s 9ms/step - loss: 0.0873
Epoch 5/100
430/430 [==============================] - 5s 13ms/step - loss: 0.0850
Epoch 6/100
430/430 [==============================] - 5s 11ms/step - loss: 0.0849
Epoch 7/100
430/430 [==============================] - 5s 12ms/step - loss: 0.0841
Epoch 8/100
430/430 [==============================] - 4s 9ms/step - loss: 0.0842
Epoch 9/100
430/430 [==============================] - 5s 12ms/step - loss: 0.0830
Epoch 10/100
430/430 [==============================] - 4s 10ms/step - loss: 0.0839
Epoch 11/100
430/430 [==============================] - 4s 9ms/step - loss: 0.0820
Epoch 12/100
430/430 [==============================] - 4s 9ms/step - loss: 0.0815
Epoc

In [6]:
predictions.shape

(3438, 1)

In [7]:
actual = scaler_y.inverse_transform(y_test)

In [8]:
actual.shape

(3438, 1)

In [9]:
prediction = scaler_y.inverse_transform(predictions)

In [10]:
x= prediction
z= actual
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE

max = x.max()
min = x.min()
mean = x.mean()
print("Maximum:",max)
print('Minimum:',min)
print("Mean:",mean)
NRMSE= (RMSE/(max-min))*100
print('Normalized Root Mean Square Error is:',NRMSE)
print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)

Maximum: 33.657665
Minimum: 0.42314208
Mean: 11.310497
Normalized Root Mean Square Error is: 11.415835808609097
Root Mean Square Error: 3.7939986657209106
Mean Square Error: 14.39442587549205
Mean Absolute Error: 2.3324853852269936


In [11]:
Calculated = pd.DataFrame(prediction, columns = ['Prediction'])
Actual = pd.DataFrame(actual, columns = ['Actual'])
combined_df = pd.concat([Calculated, Actual], axis=1)
combined_df.to_csv(r'C:\Users\sunrise\Desktop\Wind power forecasting\Bruska\Transformers\Informer_output.csv', index = False)